Note, before you can run anything involving the 1x1 moment you'll need to run the script `w3j.py`. This will generate a numpy file that stores the w3j coefficients and is loaded in to the 1x1 moment to speed up calculation. The `w3j.npy` file is too large to push to GitHib so you'll have to build it locally. Make sure that the max $\ell$ used to build the w3j numpy file matches the max $\ell$ being used elsewhere in your calculations, otherwise you run into problems (currently everything is setup to a default of $\ell_{\rm max}=384$ corresponding to nside=128.

# foreground_functions.py contents
Preamble bit:
The value of `crit` is currently set to make a beta_map with a standard deviation one thrid of the critical $\delta\beta$ for a frequency range of one decade (30-300 GHz). The `nu_0` parameter is taken at 95 GHz as the optimal pivot frequency for 30-300 GHz.

Some basic functions to build SEDs for synch and dust, and powerlaws.

## Map functions
`map_amp(ells, params)` 
- where `params = A, alpha`. 
- Generates the synfast map corresponding to a powerlaw with those parameters. 

`map_beta(ells, params)` 
- where `params = [beta, gamma]`. 
- Generates a beta_map with standard deviation set to be 1/3 times the critical value ofr convergence defined in preamble. Since the standard deviation is a function of `gamma` I've empircally fit a curve for the standard deviation at values of `gamma` so as to avoid fluctuations due to different realisations of the map when model fitting. 

`bcls(ells, params)` 
- where `params = [beta, gamma]`. Same as `map_beta` but only returning the $C_\ell$s, not the map. 

`map_const_beta_<x>(ells, freqs, params)` 
- where `<x>` is either 'synch', 'dust', or 'fg' (for foreground), `params=[A, alpha, beta]` (in 'fg' case feed parameters for synch, then for dust `params = params_s + params_d`). 
- Returns a set of frequency maps with a constant $\beta$ on the sky.

`map_<x>(ells, freqs, params)` 
- where `<x>` is either 'synch', 'dust', or 'fg' (for foreground), `params=[A, alpha, beta, gamma]` (in 'fg' case feed parameters for synch, then for dust `params = params_s + params_d`). 
- Returns a set of frequency maps with a varying $\beta$ on the sky.

From now on `<x>` is either 'synch', 'dust', or 'fg' (for foreground), `params=[A, alpha, beta, gamma]` (in 'fg' case feed parameters for synch, then for dust `params = params_s + params_d`) everywhere. The only case where the parameters aren't like this are some of the early functions which don't need gamma.

## Power spectrum functions
`ps_data_<x>(ells, freqs, params)` 
- Generates the power spectrum relating to the `<x>` component for the varying $\beta$ case. Note that these are generated at twice the nside as the maps are, to avoid weird cut off things that we don't understand when we make the model. (Basically when we extract the powerspectrum from a finite map, it doesn't match the model made with the exact same parameters at high $\ell$. However, if we generate the data at a higher nside then the deviation doesn't become relevant until $\ell$ scales we aren't looking at).

`const_ps_data_<x>(ells, freqs, params)`
- same as above, but for case of constant beta (still put gamma in as a parameter, it just gets ignored by the function).

## Moment related functions
`auto0x0_<x>(ells, freqs, params)`
- generates the 0x0 moment for these parameters, assuming autocorrolation map only and powerlaws for everything.

`auto1x1_<x>(ells, freqs, params)`
- generates the 1x1 moment for these parameters, assuming autocorrolation map only and powerlaws for everything.

`auto0x2_<x>(ells, freqs, params)`
- generates the 0x2 moment for these parameters, assuming autocorrolation map only and powerlaws for everything.

`get_wigner_sum(ells, params)`
- intermediate step needed for the 1x1 moment, is the slowest part of model calculation, and the whole reason that this is slow.

`model_<x>(ells, freqs, params)`
- generates the full model (0x0 + 1x1 + 0x2) for component `<x>`.

## Objective functions
`chi2_<x>(params, ells, freqs, data)`
- defines the $\chi^2$ for fitting with component `<x>`. We ignore the first 30 $\ell$ and use cosmic variance as our standard deviation for the moment. It's possible we could modify the objective function to penalise high $\ell$ for these finite map effects to avoid having to generate the data at twice the nside.